### Ques 1:

## Random Forest Classifier With Pipeline And Hyperparameter Tuning

In [1]:
import seaborn as sns
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
df.isnull().sum()
# df['time'].unique()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [3]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [5]:
df['time']

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [6]:
## Independent and Dependent Feature
X = df.drop(labels=['time'],axis=1)
y = df.time

In [7]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## Handle Missing Values
from sklearn.preprocessing import OneHotEncoder ## Categorical to Numerical
from sklearn.preprocessing import StandardScaler ## Feature Scaling
from sklearn.compose import ColumnTransformer

In [10]:
categorical_cols = ['sex', 'smoker', 'day']
numerical_cols = ['total_bill', 'tip', 'size']

### Feature Engineering Automation 

In [11]:
## Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), ## Handling missing values
        ('scalter',StandardScaler())  ## Feature scaling
    ]
)
## Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')), ## Handling Missing Values
        ('onehotencoder',OneHotEncoder()) ## Categorical Features to Numerical Features
    ]
)

In [12]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [13]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [14]:
X_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

### Model Training Automation

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [16]:
models = {
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier()
}

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
def evaluate_models(X_train,y_train,X_test,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        ## Train Model
        model.fit(X_train,y_train)
        
        ## Predict Testing Data
        y_test_pred = model.predict(X_test)
        
        ## Get accuracy of test data prediction
        test_model_score = accuracy_score(y_test,y_test_pred)
        
        report[list(models.keys())[i]] = test_model_score
        
    return report

In [19]:
evaluate_models(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [20]:
classifier = RandomForestClassifier()

In [21]:
## Hyperparameter Tuning
params = {
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

In [22]:
from sklearn.model_selection import RandomizedSearchCV

In [23]:
cv = RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3,)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.974 total time=   0.5s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.949 total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.974 total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.923 total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.949 total time=   0.5s
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.923 tot

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [24]:
cv.best_params_

{'n_estimators': 200, 'max_depth': None, 'criterion': 'entropy'}

In [25]:
classifier1 = RandomForestClassifier(n_estimators=100, max_depth=3, criterion='gini')

In [26]:
classifier1.fit(X_train,y_train)

RandomForestClassifier(max_depth=3)

In [27]:
y_pred = classifier1.predict(X_test)

In [28]:
accuracy_score(y_test,y_pred)

1.0

## Ques 2:

In [29]:
import seaborn as sns
df_iris = sns.load_dataset('iris')

In [30]:
df_iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [33]:
df_iris.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [34]:
df_iris.describe()

,sepal_length,sepal_width,petal_length,petal_width,species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [31]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df_iris['species'] = encoder.fit_transform(df_iris['species'])

In [32]:
df_iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [36]:
## Independent and Dependent Feature
X1 = df_iris.drop(labels=['species'],axis=1)
y1 = df_iris.species

In [39]:
X1.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [41]:
y1.tail()

145    2
146    2
147    2
148    2
149    2
Name: species, dtype: int64

In [42]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,random_state=42,test_size=0.2)

## Model Training Automation

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [44]:
models1 = {
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression()
}

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
def evaluate_models1(X_train1,y_train1,X_test1,y_test1,models1):
    
    report1 = {}
    for i in range(len(models1)):
        model = list(models1.values())[i]
        ## Train Model
        model.fit(X_train1,y_train1)
        
        ## Predict Testing Data
        y_test_pred1 = model1.predict(X_test1)
        
        ## Get accuracy of test data prediction
        test_model_score1 = accuracy_score(y_test1,y_test_pred1)
        
        report1[list(models.keys())[i]] = test_model_score1
        
    return report1

In [47]:
evaluate_models(X_train1,y_train1,X_test1,y_test1,models1)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Random Forest': 1.0, 'Logistic Regression': 1.0}